<a href="https://colab.research.google.com/github/aadityane93/SortingLine-ml/blob/main/Feature_Selection_and_Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tsfresh
from google.colab import files
import numpy as np
import pandas as pd
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_selection.relevance import calculate_relevance_table
from sklearn.model_selection import train_test_split, KFold, cross_val_predict, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, precision_score
import xgboost as xgb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.5/176.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 31.0 MB/s eta 0:00:00


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
# folder_id = '1fX9iKRJcP8UwyI_xhI-TlJdxQj1NzvBz' # 20% files
# folder_id ='1WFFJcTKOLcoFFzzm29Ch33qotT2h5gX3' #50% files
folder_id ='1lcYHGesfAhSlpaJu3CwDWo65R6bnaqCo' #80% files

folder_list = drive.ListFile({'q': "'{}' in parents and mimeType='application/vnd.google-apps.folder' and trashed=false".format(folder_id)}).GetList()


for folder in folder_list:
    folder_title = folder['title']
    print(f"Processing folder: {folder_title}")
    file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(folder['id'])}).GetList()

    for index, file in enumerate(file_list):
        file_title = file['title']
        print(f"Downloading {file_title}")
        file.GetContentFile(file_title)

Processing folder: 800hz extracted
Processing folder: 400hz extracted
Processing folder: 100hz extracted


In [ ]:
hz = 100
percent = 80

In [ ]:
def read_file(filename):
  return pd.read_csv(filename, sep=',')

extracted_train_feature = pd.DataFrame(read_file(f'extracted-{hz}hz-train-feature-{percent}percent.csv'))
extracted_test_feature = pd.DataFrame(read_file(f'extracted-{hz}hz-test-feature-{percent}percent.csv'))
extracted_train_target = pd.DataFrame(read_file(f'extracted-{hz}hz-train-target-{percent}percent.csv'))
extracted_test_target = pd.DataFrame(read_file(f'extracted-{hz}hz-test-target-{percent}percent.csv'))

In [ ]:
extracted_train_target = extracted_train_target.drop(columns=['Unnamed: 0'])
extracted_test_target = extracted_test_target.drop(columns=['Unnamed: 0'])
extracted_train_feature = extracted_train_feature.drop(columns=['Unnamed: 0'])
extracted_test_feature = extracted_test_feature.drop(columns=['Unnamed: 0'])

In [ ]:
extracted_train_feature.head()

,I_In__variance_larger_than_standard_deviation,I_In__has_duplicate_max,I_In__has_duplicate_min,I_In__has_duplicate,I_In__sum_values,I_In__abs_energy,I_In__mean_abs_change,I_In__mean_change,I_In__mean_second_derivative_central,I_In__median,...,Temp(°C)__fourier_entropy__bins_5,Temp(°C)__fourier_entropy__bins_10,Temp(°C)__fourier_entropy__bins_100,Temp(°C)__permutation_entropy__dimension_3__tau_1,Temp(°C)__permutation_entropy__dimension_4__tau_1,Temp(°C)__permutation_entropy__dimension_5__tau_1,Temp(°C)__permutation_entropy__dimension_6__tau_1,Temp(°C)__permutation_entropy__dimension_7__tau_1,Temp(°C)__query_similarity_count__query_None__threshold_0.0,Temp(°C)__mean_n_absolute_max__number_of_maxima_7
0,0.0,1.0,1.0,1.0,250.81,101.4769,0.038292,0.000415,0.000000,0.37,...,1.280506,1.919827,3.842121,1.386197,2.285083,3.157043,3.966588,4.670473,0.0,35.13
1,0.0,0.0,0.0,1.0,250.05,102.0841,0.042379,0.000389,0.000047,0.37,...,1.123708,1.765862,3.701585,1.398722,2.307939,3.216146,4.018596,4.716842,0.0,35.13
2,0.0,0.0,1.0,1.0,231.03,94.2721,0.041493,0.000520,-0.000067,0.37,...,1.092491,1.741591,3.702245,1.364469,2.287314,3.155369,3.939051,4.591458,0.0,35.13
3,0.0,0.0,1.0,1.0,219.25,89.8207,0.042157,0.000481,-0.000054,0.37,...,1.346895,2.015920,3.921954,1.462090,2.406550,3.360859,4.188281,4.904547,0.0,35.13
4,0.0,1.0,1.0,1.0,239.70,98.4812,0.040408,0.000408,-0.000033,0.37,...,1.158979,1.819628,3.760082,1.356556,2.270489,3.194101,4.025864,4.789695,0.0,35.13


In [ ]:
selected_features_train = select_features(extracted_train_feature, extracted_train_target['color'])

In [ ]:
selected_features_test = extracted_test_feature[selected_features_train.columns]

In [ ]:
selected_features_train

,I_In__sum_values,I_In__abs_energy,I_In__time_reversal_asymmetry_statistic__lag_3,"I_Out(A)__fft_coefficient__attr_""abs""__coeff_0",I_In__ratio_value_number_to_time_series_length,"Temp(°C)__fft_coefficient__attr_""abs""__coeff_0","I_In__agg_linear_trend__attr_""slope""__chunk_len_50__f_agg_""max""","I_In__augmented_dickey_fuller__attr_""teststat""__autolag_""AIC""","I_In__agg_linear_trend__attr_""slope""__chunk_len_50__f_agg_""mean""","I_Out(A)__fft_coefficient__attr_""abs""__coeff_16",...,Temp(°C)__ar_coefficient__coeff_3__k_10,"V_In__change_quantiles__f_agg_""var""__isabs_False__qh_0.8__ql_0.2","Temp(°C)__fft_coefficient__attr_""imag""__coeff_13",V_Out(V)__sample_entropy,Temp(°C)__autocorrelation__lag_3,Temp(°C)__ar_coefficient__coeff_1__k_10,"Temp(°C)__change_quantiles__f_agg_""var""__isabs_False__qh_1.0__ql_0.6",Temp(°C)__partial_autocorrelation__lag_3,"Temp(°C)__change_quantiles__f_agg_""var""__isabs_True__qh_0.6__ql_0.0","I_In__fft_coefficient__attr_""real""__coeff_11"
0,250.81,101.4769,0.000343,197.85,0.039939,22754.88,0.026000,-1.740150,0.012760,2.305471,...,-0.031315,0.001772,-2.090037,0.722427,-0.021407,-0.036845,0.031824,-0.021780,0.004061,-2.852808
1,250.05,102.0841,0.000296,198.61,0.043478,22512.10,0.026978,-2.056602,0.007286,2.851897,...,-0.056092,0.001902,0.284941,0.736839,-0.062171,-0.055457,0.032297,-0.059312,0.004071,-3.274914
2,231.03,94.2721,0.000156,184.07,0.045226,20863.58,0.035070,-2.539009,0.007643,1.922171,...,0.011245,0.001949,3.467623,0.763102,0.009946,-0.013318,0.030737,0.009653,0.004693,-1.486520
3,219.25,89.8207,0.000469,174.96,0.048043,19669.41,0.023497,-1.368272,0.013347,3.651518,...,-0.063950,0.002004,-1.206596,0.781012,-0.059379,-0.023486,0.000000,-0.059296,0.021270,-3.758038
4,239.70,98.4812,0.000687,189.20,0.045677,21460.76,0.027198,-1.757027,0.011936,0.888697,...,0.054316,0.001948,0.915158,0.757505,0.053027,0.010263,0.000000,0.052658,0.021395,-2.995558
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,352.18,141.8978,-0.000635,281.50,0.026345,31072.07,0.017474,-3.830732,0.003341,5.508957,...,0.013752,0.001864,2.794613,0.764179,0.021452,-0.047217,0.002613,0.023563,0.008027,-4.802421
3196,329.48,133.5894,-0.000389,261.90,0.033136,28817.15,0.025098,-3.849984,0.005104,2.953059,...,-0.023737,0.001871,-3.495870,0.744144,-0.023646,0.002540,0.001119,-0.023950,0.007914,-5.250496
3197,330.26,132.7940,0.000345,265.31,0.030516,29060.49,0.014923,-1.391451,0.006715,3.562817,...,-0.013506,0.002003,1.321490,0.755747,-0.011817,0.050926,0.001807,-0.014866,0.006699,-0.723433
3198,324.97,132.7549,0.000367,256.86,0.031250,28373.59,0.028946,-1.544809,0.007088,3.034836,...,0.001024,0.001831,-2.024196,0.756587,0.001210,-0.073652,0.000791,0.003202,0.008570,-1.393091


In [ ]:
selected_features_test

,I_In__sum_values,I_In__abs_energy,I_In__time_reversal_asymmetry_statistic__lag_3,"I_Out(A)__fft_coefficient__attr_""abs""__coeff_0",I_In__ratio_value_number_to_time_series_length,"Temp(°C)__fft_coefficient__attr_""abs""__coeff_0","I_In__agg_linear_trend__attr_""slope""__chunk_len_50__f_agg_""max""","I_In__augmented_dickey_fuller__attr_""teststat""__autolag_""AIC""","I_In__agg_linear_trend__attr_""slope""__chunk_len_50__f_agg_""mean""","I_Out(A)__fft_coefficient__attr_""abs""__coeff_16",...,Temp(°C)__ar_coefficient__coeff_3__k_10,"V_In__change_quantiles__f_agg_""var""__isabs_False__qh_0.8__ql_0.2","Temp(°C)__fft_coefficient__attr_""imag""__coeff_13",V_Out(V)__sample_entropy,Temp(°C)__autocorrelation__lag_3,Temp(°C)__ar_coefficient__coeff_1__k_10,"Temp(°C)__change_quantiles__f_agg_""var""__isabs_False__qh_1.0__ql_0.6",Temp(°C)__partial_autocorrelation__lag_3,"Temp(°C)__change_quantiles__f_agg_""var""__isabs_True__qh_0.6__ql_0.0","I_In__fft_coefficient__attr_""real""__coeff_11"
0,244.63,99.5949,0.000238,196.57,0.038035,19627.52,0.027527,-1.503892,0.009626,1.531561,...,-0.001080,0.001975,0.397441,0.761664,0.000752,0.009768,0.002762,-0.001928,0.009220,-1.648442
1,219.19,89.9539,0.000573,173.89,0.044563,17439.16,0.033357,-2.044660,0.012501,1.649106,...,-0.022999,0.002080,-2.440609,0.735508,-0.029714,0.023240,0.000000,-0.027884,0.014891,-2.168274
2,242.14,98.3714,0.000372,190.56,0.040064,19384.97,0.024011,-1.823773,0.009322,1.477695,...,-0.010411,0.002027,0.549680,0.763792,-0.000969,-0.092255,0.000000,-0.010153,0.021612,-1.106242
3,244.42,99.4244,0.000245,193.35,0.039683,19577.13,0.022967,-2.944703,0.010532,0.466226,...,0.003121,0.002113,-3.944461,0.720953,0.005326,-0.045505,0.000000,0.006553,0.019317,-1.387736
4,230.04,93.4120,0.000845,181.66,0.042373,18324.06,0.024476,-1.207636,0.008571,0.309310,...,0.037390,0.001930,-3.355564,0.757668,0.035559,0.026208,0.000000,0.036123,0.021710,-0.096780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,278.04,88.2192,-0.000217,282.74,0.026345,34539.93,0.016544,-4.175191,0.002099,4.257183,...,-0.032330,0.002503,0.546714,0.819172,-0.033761,0.025565,0.022531,-0.032539,0.008790,-3.936963
796,277.34,88.2342,-0.000186,282.23,0.025358,34393.41,0.008842,-4.173918,0.001637,3.864915,...,0.009065,0.002828,0.984551,0.793951,0.011383,0.022293,0.022579,0.009623,0.009298,3.283723
797,267.63,85.0573,-0.000192,271.06,0.022883,33144.87,0.013674,-3.784736,0.002073,7.058504,...,0.065882,0.002759,-0.595460,0.848634,0.067967,0.049938,0.021879,0.067892,0.008816,1.907506
798,263.57,83.2995,-0.000226,270.40,0.025434,32812.70,0.008452,-4.338074,0.001971,4.941128,...,-0.033031,0.002806,-0.018340,0.831808,-0.034348,-0.036394,0.025342,-0.036872,0.006654,3.415806


In [ ]:
extracted_test_target

,color
0,0
1,0
2,0
3,0
4,0
...,...
795,3
796,3
797,3
798,3


In [ ]:
model = RandomForestClassifier()
model.fit(selected_features_train,extracted_train_target)
y_pred = model.predict(selected_features_test)
accuracy = accuracy_score(extracted_test_target, y_pred)
precision = precision_score(extracted_test_target, y_pred, average='weighted')
print(classification_report(extracted_test_target, y_pred))
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(confusion_matrix(extracted_test_target, y_pred))

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


              precision    recall  f1-score   support

           0       0.99      0.97      0.98       200
           1       0.44      0.62      0.52       200
           2       0.87      0.17      0.28       200
           3       0.70      1.00      0.82       200

    accuracy                           0.69       800
   macro avg       0.75      0.69      0.65       800
weighted avg       0.75      0.69      0.65       800

Accuracy: 0.6887
Precision: 0.7499
[[194   1   5   0]
 [  0 124   0  76]
 [  2 154  33  11]
 [  0   0   0 200]]


In [ ]:
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, precision_score

final_model = xgb.XGBClassifier(
    # objective="multi:softmax",
    # random_state=42,
    alpha=10,
    reg_lambda=1,
    # max_depth=10,
    min_child_weight=10,
    # subsample=0.8,
    colsample_bytree=0.8
)

final_model.fit(selected_features_train,extracted_train_target)
y_pred = final_model.predict(selected_features_test)
accuracy = accuracy_score(extracted_test_target, y_pred)
precision = precision_score(extracted_test_target, y_pred, average='weighted')
print(classification_report(extracted_test_target, y_pred))
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(confusion_matrix(extracted_test_target, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99       200
           1       0.99      0.94      0.96       200
           2       1.00      0.97      0.99       200
           3       0.92      0.99      0.95       200

    accuracy                           0.97       800
   macro avg       0.98      0.97      0.98       800
weighted avg       0.98      0.97      0.98       800

Accuracy: 0.9750
Precision: 0.9766
[[198   0   0   2]
 [  0 188   0  12]
 [  0   1 195   4]
 [  0   1   0 199]]
